In [3]:
!pip install joblib


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [35]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
from playstore.config import CLEAN_DATA_DIR

In [36]:
# Load data
try:
    df = pd.read_csv("cleaned_googleplaystore.csv")  # Update to your actual file path if different
    print("Loaded")
except Exception as e:
    print("Error loading data:", e)
    df = pd.DataFrame({
        'Category': ['Game', 'Education', 'Business'],
        'Rating': [4.5, 4.0, 3.8],
        'Type': ['Free', 'Free', 'Paid'],
        'Installs': [1000000, 500000, 100000],
    })



Loaded


In [40]:
# Select numeric columns
datanum = [col for col in df.columns if df[col].dtype != 'O']
dfnum = df[datanum]

# Define functions for outlier handling
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.10)
    quartile3 = dataframe[variable].quantile(0.90)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(df, numeric_columns):
    for variable in numeric_columns:
        low_limit, up_limit = outlier_thresholds(df, variable)
        df.loc[(df[variable] < low_limit), variable] = low_limit
        df.loc[(df[variable] > up_limit), variable] = up_limit

# Handle outliers
replace_with_thresholds(dfnum, dfnum.columns)

# Prepare data for prediction
y = df["Rating"]
x = dfnum
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.81, random_state=81)

# Train and evaluate model
rf = RandomForestRegressor()
model = rf.fit(x_train, y_train)
score = model.score(x_test, y_test)

print("Model Score:", score)

/var/folders/kd/bldn4b0j3q31pb8c56pgmyy80000gn/T/ipykernel_60493/3917860194.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-823644.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[(df[variable] < low_limit), variable] = low_limit
/var/folders/kd/bldn4b0j3q31pb8c56pgmyy80000gn/T/ipykernel_60493/3917860194.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-7.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[(df[variable] < low_limit), variable] = low_limit


Model Score: 0.9984980763593673


In [38]:
model_path = os.path.join(CLEAN_DATA_DIR, "rf_model.pkl")
joblib.dump(model, "rf_model.pkl")

['rf_model.pkl']

In [ ]:
model_path = os.path.join(CLEAN_DATA_DIR, "rf_model2.pkl")
joblib.dump(model, "rf_model2.pkl")